<h1 align="center">GENERATE INSTAGRAM IMAGES</h1>


In [ ]:
from PIL import Image

## 1 - OPEN IMAGES

In [ ]:
ref_path = "data/six_pub_example.jpg"
ref = Image.open(ref_path)
print(ref.format, ref.size, ref.mode)

In [ ]:
base_path = "data/base.jpg"
base = Image.open(base_path)
print(base.format, base.size, base.mode)

In [ ]:
aum_path = "data/aum.webp"
aum = Image.open(aum_path)
print(aum.format, aum.size, aum.mode)

## 2 - CREATE BASE IMAGE

In [ ]:
# --- Utilisation ---
# - conserve la hauteur de ref
# - pas de déformation
# - décalage horizontal de offset_x px avant crop
from utils import resize_match_ref_height_and_crop

resized = resize_match_ref_height_and_crop(
    base, ref,
    offset_x=60,     # décale la fenêtre de crop vers la droite
    offset_y=0,      # laisse 0 (on a déjà la bonne hauteur)
    resample=Image.Resampling.BICUBIC,
    pad_if_narrow=True,     # si base_redim est plus étroite que ref, on pad
    pad_fill=(0,0,0,0)      # fond transparent si RGBA, sinon mets un RGB
)

# result a exactement la taille de ref.size
resized.save("results/align_ref_height.png")
print(resized.size)


In [ ]:
from utils import upscale_preserve_aspect

resized_big = upscale_preserve_aspect(resized, scale=2.0)
resized_big.save("results/resized_big.png")

## 3 - CREATE WATERMARK

In [ ]:
from utils import select_black_pixels

blacked = select_black_pixels(
    img=aum,
    r_max=50,
    g_max=50,
    b_max=50,
    mode="mask"
)
blacked.save(f"results/blacked_aum.png")

## 4 - APPLY WATERMARK

In [ ]:
from utils import subtract_constant_clip0, add_constant_clip255
from utils import subtract_images

out = subtract_images(resized_big, blacked, match_size=True)
out.save("results/sub_L.png")

In [ ]:
from utils import add_images

out = add_images(resized_big, blacked, match_size=True)
out.save("results/add_L.png")

## 5 - SPLIT IN TILES

In [ ]:
from utils import split_grid

tiles = split_grid(out, rows=2, cols=3)

# Sauvegarder chaque tuile
for i, t in enumerate(tiles):
    t.save(f"tiles/tile_{i:02d}.png")

<h1 align="center">GENERATE INSTAGRAM TILES</h1>

In [1]:
from PIL import Image
from utils import   resize_match_ref_height_and_crop, \
                    upscale_preserve_aspect, \
                    select_black_pixels, \
                    subtract_constant_clip0, \
                    add_constant_clip255, \
                    subtract_images, \
                    add_images, \
                    split_grid, \
                    grey_mask, \
                    save_tiles
                    

In [2]:
ref_path = "data/six_pub_example.jpg"
base_path = "data/stones.jpg"
wmark_path = "data/aum2.jpg"

ref = Image.open(ref_path)
base = Image.open(base_path)
wmark = Image.open(wmark_path)

In [ ]:
# config base transform
offset_x=60
offset_y=0
resample=Image.Resampling.BICUBIC
scale=2.0

# config watermark
r_max=50
g_max=50
b_max=50

# mode
mask_mode="grey_watermark" # dark_watermark, light_watermark, grey_watermark
k=180
split_color_mode=None # None, grey_even

In [ ]:
resized = resize_match_ref_height_and_crop(
    base, ref,
    offset_x=offset_x,     # décale la fenêtre de crop vers la droite
    offset_y=offset_y,      # laisse 0 (on a déjà la bonne hauteur)
    resample=resample,
    pad_if_narrow=True,     # si base_redim est plus étroite que ref, on pad
    pad_fill=(0,0,0,0)      # fond transparent si RGBA, sinon mets un RGB
)

resized_big = upscale_preserve_aspect(resized, scale=scale)

#resized_big = resized_big.quantize(colors=32, method=Image.Quantize.MEDIANCUT, kmeans=3)
resized_big = resized_big.quantize(colors=16, dither=Image.Dither.FLOYDSTEINBERG)

blacked = select_black_pixels(
    img=wmark,
    r_max=150,
    g_max=150,
    b_max=150,
    mode="mask"
)

if mask_mode == "dark_watermark":
    out = subtract_images(resized_big, blacked, match_size=True, k=k)
elif mask_mode == "light_watermark":
    out = add_images(resized_big, blacked, match_size=True, k=k)
else:
    out = grey_mask(
        resized_big, blacked,
        threshold=128, feather=2.0,
        invert_mask=True,
        outline_width=1, outline_color=(0, 0, 0), 
        outline_opacity=220, 
        outline_feather=1.0
    )

tiles = split_grid(out, rows=2, cols=3)

save_tiles(out=out, tiles=tiles, folder='tiles', mask_mode=mask_mode, split_color_mode=split_color_mode)